## LSTM на оригинальном датасете

Попытка сделать монофонический выход из сетки

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

Сделаем также пользовательский импорт

In [41]:
from decode_patterns import data_conversion

Загружаем датасет

In [60]:
# import dataset
drum, bass = data_conversion.make_lstm_dataset(height=16, limit=20000, patterns_file="decode_patterns/patterns.pairs.tsv", mono=True)


# define shuffling of dataset
def shuffle(A, B, p=0.8):
    # take 80% to training, other to testing
    L = len(A)
    idx = np.arange(L) < p*L
    np.random.shuffle(idx)
    yield A[idx]
    yield B[idx]
    yield A[np.logical_not(idx)]
    yield B[np.logical_not(idx)]
    
    
# we can select here a validation set
drum, bass, drum_validation, bass_validation = shuffle(drum, bass)
    
# and we can shuffle train and test set like this:
# drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)

In [62]:
bass_validation[9]

array([1, 1, 8, 1, 8, 1, 1, 1, 8, 1, 8, 1, 8, 1, 1, 6], dtype=int64)

Модель определим в самом простом варианте, который только можно себе представить -- как в примере с конечным автоматом

In [63]:
# попробуем определить модель LSTM как конечный автомат
class DrumNBassLSTM(nn.Module):
    def __init__(self):
        super(DrumNBassLSTM, self).__init__()
        # one input neuron, one output neuron, one layer in LSTM block
        self.input_size = 14
        self.hidden_size = 34
        self.layer_count = 1
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.layer_count)
        self.embed_layer = nn.Linear(self.hidden_size, 1)
        self.sigm = nn.Sigmoid()
    
    def forward(self, input):
        # пусть в input у нас приходит вектор размерности (64, 32, 14)
        # то есть 64 отсчёта, тридцать два примера (минибатч), 14 значение в каждом (барабанная партия)
        output, _ = self.lstm(input)
        output = self.sigm(self.embed_layer(output))*37
        return output

In [64]:
# часть обучения
dnb_lstm = DrumNBassLSTM()

criterion = nn.MSELoss()

# оценим также и разнообразие мелодии по её.. дисперсии?)
# def melody_variety(melody):
#     return 1/(1 + (melody.sum(axis=2) > 1).int())
    
# criterion = nn.NLLLoss() # -- этот товарищ требует, чтобы LSTM выдавал классы,
# criterion = nn.CrossEntropyLoss() # и этот тоже
# (числа от 0 до C-1), но как всё-таки его заставить это делать?...
# optimizer = optim.SGD(dnb_lstm.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(dnb_lstm.parameters(), lr=0.001)

Найденные баги и их решения:

https://stackoverflow.com/questions/56741087/how-to-fix-runtimeerror-expected-object-of-scalar-type-float-but-got-scalar-typ

https://stackoverflow.com/questions/49206550/pytorch-error-multi-target-not-supported-in-crossentropyloss/49209628

https://stackoverflow.com/questions/56243672/expected-target-size-50-88-got-torch-size50-288-88

In [65]:
epoch_count = 500
batch_size = 128
shuffle_every_epoch = True
    
if shuffle_every_epoch:
    print(f"shuffle_every_epoch is on")
else:
    print(f"shuffle_every_epoch is off")
    # shuffle train and test set:
    drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)
    drum_train = torch.tensor(drum_train, dtype=torch.float)
    bass_train = torch.tensor(bass_train, dtype=torch.float)
    drum_test = torch.tensor(drum_test, dtype=torch.float)
    drum_test = torch.tensor(drum_test, dtype=torch.float)
        
for epoch in range(epoch_count):  # loop over the dataset multiple times
    print(f"Epoch #{epoch}")
    if shuffle_every_epoch:
        # shuffle train and test set:
        drum_train, bass_train, drum_test, bass_test = shuffle(drum, bass)
        drum_train = torch.tensor(drum_train, dtype=torch.float)
        bass_train = torch.tensor(bass_train, dtype=torch.float)
        drum_test = torch.tensor(drum_test, dtype=torch.float)
        bass_test = torch.tensor(bass_test, dtype=torch.float)
        
    examples_count = drum_train.size()[0]
    examples_id = 0
    
    running_loss = 0.0
    runnint_count = 0
    batch_id = 0
    while examples_id < examples_count:
        batch_drum_train = drum_train[examples_id:examples_id + batch_size,:,:].transpose(0,1)
        batch_bass_train = bass_train[examples_id:examples_id + batch_size,].transpose(0,1)
        # transpose нужен для обмена размерности батча и размерности шагов
#         print(f"batch_drum_train:{batch_drum_train.size()}, batch_bass_train:{batch_bass_train.size()}")

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        bass_outputs = dnb_lstm(batch_drum_train).squeeze()
#         bass_outputs = bass_outputs.reshape(bass_outputs.size()[0], -1)
#         batch_bass_train = batch_bass_train.reshape(batch_bass_train.size()[0], -1)
#         print(f"bass_outputs:{bass_outputs.size()} batch_bass_train: {batch_bass_train.size()}")
#         print(f"bass_outputs:{bass_outputs} batch_bass_train: {batch_bass_train}")
        
        # loss = criterion(bass_outputs, batch_bass_train.long())
        loss = criterion(bass_outputs, batch_bass_train)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        runnint_count += 1
        period = 5
        if batch_id % period == 0 or examples_id + batch_size >= examples_count:
            print('[%d, %5d] train loss: %.7f' %
                  (epoch + 1, batch_id + 1, running_loss / runnint_count))
            running_loss = 0.0
            runnint_count = 1
            
        # update batch info
        examples_id += batch_size
        batch_id += 1
        
    # here we can insert measure error on test set

#should check accuracy on validation set
print('Finished Training')

shuffle_every_epoch is on
Epoch #0
[1,     1] train loss: 176.1044464
[1,     6] train loss: 134.1047796
[1,    11] train loss: 112.3866145
[1,    16] train loss: 102.9875336
[1,    21] train loss: 83.1650174
[1,    26] train loss: 82.2173220
[1,    31] train loss: 80.3045235
[1,    36] train loss: 65.7691758
[1,    41] train loss: 57.2387377
[1,    46] train loss: 61.9482002
[1,    51] train loss: 59.1013546
[1,    56] train loss: 57.0269629
[1,    61] train loss: 61.0347366
[1,    66] train loss: 53.9613469
[1,    71] train loss: 57.5882696
[1,    76] train loss: 47.0909729
[1,    81] train loss: 56.3308563
[1,    86] train loss: 54.4532382
[1,    91] train loss: 54.6777617
[1,    96] train loss: 44.8776766
[1,   100] train loss: 40.2034042
Epoch #1
[2,     1] train loss: 53.6293182
[2,     6] train loss: 50.2689387
[2,    11] train loss: 48.4459623
[2,    16] train loss: 43.8645871
[2,    21] train loss: 47.1056913
[2,    26] train loss: 51.5786514
[2,    31] train loss: 56.1698812


[12,    21] train loss: 44.3402627
[12,    26] train loss: 44.6274217
[12,    31] train loss: 52.3281593
[12,    36] train loss: 41.8031629
[12,    41] train loss: 40.1266454
[12,    46] train loss: 51.8167197
[12,    51] train loss: 45.1618252
[12,    56] train loss: 46.3268846
[12,    61] train loss: 50.2223568
[12,    66] train loss: 43.5352039
[12,    71] train loss: 47.3148524
[12,    76] train loss: 37.2679571
[12,    81] train loss: 44.4215654
[12,    86] train loss: 41.8869864
[12,    91] train loss: 49.1308041
[12,    96] train loss: 38.1630936
[12,   100] train loss: 35.3949478
Epoch #12
[13,     1] train loss: 38.9863167
[13,     6] train loss: 42.0444285
[13,    11] train loss: 42.1489474
[13,    16] train loss: 40.1826706
[13,    21] train loss: 44.5478681
[13,    26] train loss: 42.2498932
[13,    31] train loss: 52.8089472
[13,    36] train loss: 40.4510848
[13,    41] train loss: 40.9620304
[13,    46] train loss: 51.5613575
[13,    51] train loss: 46.7151445
[13,    56

[23,    16] train loss: 38.1869322
[23,    21] train loss: 42.8718580
[23,    26] train loss: 43.8489615
[23,    31] train loss: 50.9373951
[23,    36] train loss: 41.1116994
[23,    41] train loss: 42.8950933
[23,    46] train loss: 50.4388205
[23,    51] train loss: 45.7246030
[23,    56] train loss: 44.6139431
[23,    61] train loss: 50.7990557
[23,    66] train loss: 43.2851473
[23,    71] train loss: 46.9392719
[23,    76] train loss: 36.2552357
[23,    81] train loss: 43.0326042
[23,    86] train loss: 40.8723571
[23,    91] train loss: 46.4214846
[23,    96] train loss: 38.0721359
[23,   100] train loss: 36.5688362
Epoch #23
[24,     1] train loss: 43.9016647
[24,     6] train loss: 42.0432288
[24,    11] train loss: 43.7694295
[24,    16] train loss: 39.1246662
[24,    21] train loss: 43.4920769
[24,    26] train loss: 44.4759998
[24,    31] train loss: 52.0731907
[24,    36] train loss: 40.9975675
[24,    41] train loss: 41.9906851
[24,    46] train loss: 50.6954937
[24,    51

[34,    11] train loss: 43.9502741
[34,    16] train loss: 39.6971773
[34,    21] train loss: 42.4660403
[34,    26] train loss: 43.0779858
[34,    31] train loss: 52.2226969
[34,    36] train loss: 40.2930298
[34,    41] train loss: 40.5722446
[34,    46] train loss: 52.1829103
[34,    51] train loss: 45.3144951
[34,    56] train loss: 45.1415869
[34,    61] train loss: 50.6950518
[34,    66] train loss: 42.7962551
[34,    71] train loss: 45.5707397
[34,    76] train loss: 34.8556226
[34,    81] train loss: 41.7032026
[34,    86] train loss: 39.7733237
[34,    91] train loss: 48.1709652
[34,    96] train loss: 36.1493905
[34,   100] train loss: 39.8361885
Epoch #34
[35,     1] train loss: 39.0134926
[35,     6] train loss: 39.6016401
[35,    11] train loss: 44.6101634
[35,    16] train loss: 39.0009193
[35,    21] train loss: 43.7380714
[35,    26] train loss: 42.8293540
[35,    31] train loss: 50.6206264
[35,    36] train loss: 39.5892251
[35,    41] train loss: 41.7027639
[35,    46

[45,     6] train loss: 41.9851418
[45,    11] train loss: 40.6379973
[45,    16] train loss: 38.2061844
[45,    21] train loss: 41.9210072
[45,    26] train loss: 44.1896063
[45,    31] train loss: 50.2721513
[45,    36] train loss: 41.7191614
[45,    41] train loss: 39.9165026
[45,    46] train loss: 50.4596526
[45,    51] train loss: 46.6064885
[45,    56] train loss: 47.9014289
[45,    61] train loss: 46.6831347
[45,    66] train loss: 39.9768457
[45,    71] train loss: 46.1108354
[45,    76] train loss: 35.2851394
[45,    81] train loss: 40.9162273
[45,    86] train loss: 39.7056160
[45,    91] train loss: 46.4463196
[45,    96] train loss: 37.2881845
[45,   100] train loss: 36.4625526
Epoch #45
[46,     1] train loss: 38.9828072
[46,     6] train loss: 41.8731276
[46,    11] train loss: 41.0472266
[46,    16] train loss: 39.5732721
[46,    21] train loss: 40.8256493
[46,    26] train loss: 43.5568937
[46,    31] train loss: 52.6444677
[46,    36] train loss: 41.0559298
[46,    41

...

[313,    21] train loss: 39.8627758
[313,    26] train loss: 42.2328218
[313,    31] train loss: 45.5046107
[313,    36] train loss: 37.2295494
[313,    41] train loss: 38.8162581
[313,    46] train loss: 45.8474789
[313,    51] train loss: 40.0578918
[313,    56] train loss: 38.3576056
[313,    61] train loss: 44.2213262
[313,    66] train loss: 40.7354762
[313,    71] train loss: 41.8512071
[313,    76] train loss: 30.7670212
[313,    81] train loss: 36.1177247
[313,    86] train loss: 34.8955669
[313,    91] train loss: 44.0843175
[313,    96] train loss: 32.7604701
[313,   100] train loss: 33.5401443
Epoch #313
[314,     1] train loss: 37.8987885
[314,     6] train loss: 39.3007692
[314,    11] train loss: 39.5258567
[314,    16] train loss: 38.3480428
[314,    21] train loss: 37.5430158
[314,    26] train loss: 38.6197751
[314,    31] train loss: 46.7613767
[314,    36] train loss: 36.4986814
[314,    41] train loss: 37.6144377
[314,    46] train loss: 45.2714761
[314,    51] trai

[323,    91] train loss: 43.6977857
[323,    96] train loss: 33.8623676
[323,   100] train loss: 32.9503883
Epoch #323
[324,     1] train loss: 36.4090958
[324,     6] train loss: 37.5371882
[324,    11] train loss: 37.7289429
[324,    16] train loss: 37.7416414
[324,    21] train loss: 38.7881222
[324,    26] train loss: 41.3296185
[324,    31] train loss: 44.3359299
[324,    36] train loss: 38.3018964
[324,    41] train loss: 37.7407074
[324,    46] train loss: 44.9022331
[324,    51] train loss: 40.8416131
[324,    56] train loss: 38.6462472
[324,    61] train loss: 44.8172455
[324,    66] train loss: 41.2665958
[324,    71] train loss: 40.4347649
[324,    76] train loss: 31.4336770
[324,    81] train loss: 37.2858791
[324,    86] train loss: 33.5222174
[324,    91] train loss: 42.5941982
[324,    96] train loss: 33.8299573
[324,   100] train loss: 33.3024681
Epoch #324
[325,     1] train loss: 34.3183517
[325,     6] train loss: 38.5849965
[325,    11] train loss: 37.4226023
[325, 

[334,    56] train loss: 38.2253183
[334,    61] train loss: 44.0296510
[334,    66] train loss: 38.7512296
[334,    71] train loss: 40.7995796
[334,    76] train loss: 31.4454724
[334,    81] train loss: 35.8591563
[334,    86] train loss: 35.8219322
[334,    91] train loss: 41.0776577
[334,    96] train loss: 34.5299956
[334,   100] train loss: 30.5069229
Epoch #334
[335,     1] train loss: 36.9926949
[335,     6] train loss: 39.0916456
[335,    11] train loss: 37.9644171
[335,    16] train loss: 37.6262226
[335,    21] train loss: 37.5018368
[335,    26] train loss: 41.9092890
[335,    31] train loss: 46.9774876
[335,    36] train loss: 35.3901641
[335,    41] train loss: 38.8207887
[335,    46] train loss: 46.3021463
[335,    51] train loss: 40.0096490
[335,    56] train loss: 39.7088191
[335,    61] train loss: 44.0618108
[335,    66] train loss: 38.4591096
[335,    71] train loss: 40.9851685
[335,    76] train loss: 30.6739324
[335,    81] train loss: 36.5053883
[335,    86] trai

[345,    21] train loss: 38.8683484
[345,    26] train loss: 40.1923402
[345,    31] train loss: 43.9020863
[345,    36] train loss: 34.8539880
[345,    41] train loss: 37.4468899
[345,    46] train loss: 44.8348942
[345,    51] train loss: 39.6580578
[345,    56] train loss: 38.6986777
[345,    61] train loss: 44.8187218
[345,    66] train loss: 39.5069653
[345,    71] train loss: 41.0136534
[345,    76] train loss: 31.4198847
[345,    81] train loss: 35.4463355
[345,    86] train loss: 33.9722436
[345,    91] train loss: 43.1312574
[345,    96] train loss: 33.8831679
[345,   100] train loss: 32.9759529
Epoch #345
[346,     1] train loss: 36.7666893
[346,     6] train loss: 39.0861702
[346,    11] train loss: 36.7762877
[346,    16] train loss: 38.3374666
[346,    21] train loss: 37.5529156
[346,    26] train loss: 41.2487450
[346,    31] train loss: 44.9072037
[346,    36] train loss: 36.3471813
[346,    41] train loss: 36.8718828
[346,    46] train loss: 45.0296974
[346,    51] trai

KeyboardInterrupt: 

In [51]:
batch_drum_train = drum_train[:,:,:].transpose(0,1)
batch_bass_train = bass_train[:,:].transpose(0,1)
with torch.no_grad():
    bass_outputs = dnb_lstm(batch_drum_train)

In [59]:
result = bass_outputs.squeeze().int()
result

tensor([[ 6,  3,  3,  ...,  5,  5,  5],
        [ 3,  4,  4,  ...,  5,  5,  3],
        [ 4,  1,  1,  ...,  4,  4,  3],
        ...,
        [ 1,  7,  7,  ...,  5,  5,  1],
        [ 5,  1,  1,  ...,  2,  2,  4],
        [ 1, 10, 10,  ...,  4,  4,  1]], dtype=torch.int32)

Попробуем сохранить результаты работы сети. На anaconda нет mido, поэтому сохраняем результаты работы просто в массивчик npy... Однако, как альтернатива, его можно поставить чере pip в conda:
https://github.com/mido/mido/issues/198

In [66]:
import mido
from decode_patterns.data_conversion import build_track, DrumMelodyPair, Converter


converter = Converter((16,50))

batch_drum = torch.cat((drum_train, drum_test, torch.tensor(drum_validation))).transpose(0,1)
batch_bass = torch.cat((bass_train.int(), bass_test.int(), torch.tensor(bass_validation).int())).transpose(0,1)
with torch.no_grad():
    bass_outputs = dnb_lstm(batch_drum)
    bass_outputs = bass_outputs.squeeze().int()
    
    for i in range(bass_outputs.size()[1]):
        bass_seq = bass_outputs[:,i]
#         bass_seq = batch_bass[:,i]
#         print(f"bass_seq:{bass_seq.size()}")
        bass_output = []
        for bass_note in bass_seq:
            bass_row = np.eye(1, 36, bass_note - 1)[0]
            bass_output.append(bass_row)
        bass_output = torch.tensor(bass_output).int().squeeze()
#         print(f"bass_output:{bass_output.size()}")
        
#         print(f"batch_drum:{batch_drum[:,i,:].size()}, bass_output:{bass_output.size()}")
            
        img_dnb = torch.cat((batch_drum[:,i,:].int(),bass_output), axis=1)
#         print(f"img_dnb:{list(bass_output)}")
        pair = converter.convert_numpy_image_to_pair(np.array(img_dnb))
#         print(f"pair.melody:{pair.melody}")
        mid = build_track(pair, tempo=pair.tempo)
        mid.save(f"midi/npy/sample{i+1}.mid")
#         np.save(f"midi/npy/drum{i+1}.npy", batch_drum[:,i,:].int())
#         np.save(f"midi/npy/bass{i+1}.npy", bass_outputs[:,i,:])